In [ ]:
!pip install ultralytics --upgrade
import cv2

from ultralytics import YOLO
from ultralytics import solutions

cap = cv2.VideoCapture("/content/Australian racing greyhounds  - Dog race [gEX787opZvE].webm")
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
video_writer = cv2.VideoWriter("speed_estimation.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Initialize SpeedEstimator
speedestimator = solutions.SpeedEstimator(
    region=[(0, 360), (1280, 360)],
    model="/content/best (3).pt",
    show=True,
)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        break
    results = speedestimator(im0)
    video_writer.write(results.plot_im)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

In [ ]:
!pip install ultralytics --upgrade
import cv2
import numpy as np
from ultralytics import YOLO
from ultralytics import solutions

# Load the video
video_path = "/content/Australian racing greyhounds  - Dog race [gEX787opZvE].webm"
cap = cv2.VideoCapture(video_path)

# Get video properties
w, h = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30  # Default to 30 FPS if not detected

# Video Writer
video_writer = cv2.VideoWriter("speed_estimation.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Define ROI
roi = [(100, 400), (1180, 400)]  # Adjusted for better perspective

# Initialize SpeedEstimator
speed_estimator = solutions.SpeedEstimator(
    region=roi,
    model="/content/best (3).pt",
    show=False,  
    conf=0.4,  
    analytics_type="bbox",  # Changed from 'line' to 'bbox'
)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        break

    # Apply slight Gaussian blur to reduce noise
    im0 = cv2.GaussianBlur(im0, (5, 5), 0)

    # Run speed estimation
    results = speed_estimator(im0)

    # Debugging: Print result attributes
    print(dir(results))  # Check available attributes
    print(results)  # Print structured output

    if hasattr(results, "detections"):  # Check if detections exist
        for det in results.detections:
            x1, y1, x2, y2 = map(int, det["bbox"])  
            speed = det.get("speed", 0)  # Avoid errors if speed isn't available
            label = f"{speed:.2f} km/h"

            # Draw bounding box and speed label
            cv2.rectangle(im0, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(im0, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    # Write the processed frame
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()